# Palabras clave (1ª versión)

Este es el primer cuaderno de la presentación [Text mining is Not Magic](https://docs.google.com/presentation/d/15l5MarIFXz67HqtSoqYEGLTioNt_D5F7YPuMIB4EvIU)

## Frecuencia de aparición

In [1]:
from collections import Counter

def get_keywords(words):
    '''Return the top10 frequent words.
    '''
    return [
        x for x,_ in 
        sorted(
            Counter(words).iteritems(),
            key=lambda (_,freq):freq,
            reverse=True
        )
    ][:10]

In [2]:
print get_keywords(['esta', 'es', 'una', 'prueba', 'para', 'ver', 'si', 'es', 'correcta', 'la', 'función', 'para', 'calcular', 'frecuencias'])

['para', 'es', 'ver', 'esta', 'correcta', 'frecuencias', 'la', 'calcular', 'una', 'si']


## Carga de documentos

Dataset URL: http://www.lsi.us.es/~fermin/corpusCine.zip

Nos quedamos con los ficheros .xml, que tienen una estructura del estilo:

```xml
<review author="XXX" title="XXX" rank="X" maxRank="X" source="XXX">
	<summary>XXX</summary>
	<body>XXX</body>
</review>
```

In [3]:
import os
import config

### Detección de la codificación

In [4]:
import chardet

with open(config.DATASET_MUCHOCINE_RAW+'/100.xml', 'r') as fd:
    txt = fd.read()
    
print chardet.detect(txt)

{'confidence': 0.9248548221453007, 'encoding': 'ISO-8859-2'}


### Parseado con expresiones regulares

In [5]:
import codecs  # Know your encoding
import re

def parse_file(file_path):
    retval = {'file': os.path.basename(file_path)}
    with codecs.open(file_path,'r', encoding='ISO-8859-2') as fd:
        f = fd.read().strip()
    reg_expr = re.compile(r'\<review author="(?P<author>.*)" title="(?P<title>.*)" rank="(?P<rank>\d)".*\>\s*<summary>(?P<summary>.*)</summary>\s*<body>(?P<body>.*)</body>\s*</review>')
    regexp_result = reg_expr.search(f.strip())
    for key in ['author', 'rank', 'title', 'summary', 'body']:
        retval[key] = regexp_result.group(key)
    return retval

### Carga

In [6]:
documents = []
for file_name in os.listdir(config.DATASET_MUCHOCINE_RAW):
    try:
        documents.append(
            parse_file(config.DATASET_MUCHOCINE_RAW+'/'+file_name)
        )
    except Exception as e:
        print file_name
        print e.message
documents.sort(key=lambda x: x['file'])

## Documento de ejemplo

In [7]:
my_document = documents[1478]  # 'Harry Potter y la piedra filosofal'
print my_document['title']
print my_document['body']

Harry Potter y la piedra filosofal
Antes de nada, me declaro fan incondicional de los libros de Harry y Potter. De sus películas soy un poco menos fan, aunque me he comprado el pack de DVDs con las tres primeras. Y que vergüenza al comprarlas, que me dijeron que esas las vendía sólo en la sección de nińos, que no eran películas de adultos. Pues ale, me alegro, que además me salieron más baratas que las de Star Wars.La película trata sobre un chaval que es dejado en una casa por un grupo de extrańos magos cuando no es más que un bebe. Doce ańos más tarde, el chaval ha crecido y su situación en la vida ha empeorado considerablemente. Vive debajo de la escalera en la casa de sus tíos, los cuales le odian y es vejado continuamente por su desagradable primo. Pero esta situación que tiene cambia el día que recibe una extrańa carta de un colegio para magos.La primera de la saga de Harry Potter nos presenta a los personajes de la historia de un modo paulatino y nada forzado. Tiene un guión bas

## 1ª versión. Contar palabras

In [8]:
def txt2words1(txt):
    words = [w for w in txt.split(' ') if w!='']
    return words

In [9]:
print get_keywords(txt2words1(my_document['body']))

[u'de', u'que', u'la', u'me', u'un', u'el', u'los', u'no', u'es', u'y']


## 2ª versión. Stopwords

In [10]:
## Taken from NLTK corpus so you don't have to download

#import nltk
#nltk.download()
#from nltk.corpus import stopwords
#print filter_symbols(
#    ' '.join(
#        sorted(stopwords.words('spanish'))
#    )
#)

STOPWORDS = set('''
a al algo algunas algunos ante antes como con contra cual cuando de del desde donde durante e el ella ellas ellos
en entre era erais eran eras eres es esa esas ese eso esos esta estaba estabais estaban estabas estad estada estadas
estado estados estamos estando estar estaremos estara estaran estaras estare estareis estaria estariais estariamos
estarian estarias estas este estemos esto estos estoy estuve estuviera estuvierais estuvieran estuvieras estuvieron
estuviese estuvieseis estuviesen estuvieses estuvimos estuviste estuvisteis estuvieramos estuviesemos estuvo esta
estabamos estais estan estas este esteis esten estes fue fuera fuerais fueran fueras fueron fuese fueseis fuesen fueses
fui fuimos fuiste fuisteis fueramos fuesemos ha habida habidas habido habidos habiendo habremos habra habran habras
habre habreis habria habriais habriamos habrian habrias habeis habia habiais habiamos habian habias han has hasta
hay haya hayamos hayan hayas hayais he hemos hube hubiera hubierais hubieran hubieras hubieron hubiese hubieseis
hubiesen hubieses hubimos hubiste hubisteis hubieramos hubiesemos hubo la las le les lo los me mi mis mucho muchos
muy mas mi mia mias mio mios nada ni no nos nosotras nosotros nuestra nuestras nuestro nuestros o os otra otras otro
otros para pero poco por porque que quien quienes que se sea seamos sean seas sentid sentida sentidas sentido sentidos
seremos sera seran seras sere sereis seria seriais seriamos serian serias seais siente sin sintiendo sobre sois somos
son soy su sus suya suyas suyo suyos si tambien tanto te tendremos tendra tendran tendras tendre tendreis tendria
tendriais tendriamos tendrian tendrias tened tenemos tenga tengamos tengan tengas tengo tengais tenida tenidas tenido
tenidos teniendo teneis tenia teniais teniamos tenian tenias ti tiene tienen tienes todo todos tu tus tuve tuviera
tuvierais tuvieran tuvieras tuvieron tuviese tuvieseis tuviesen tuvieses tuvimos tuviste tuvisteis tuvieramos tuviesemos
tuvo tuya tuyas tuyo tuyos tu un una uno unos vosostras vosostros vuestra vuestras vuestro vuestros y ya yo el eramos
'''.split())

def txt2words2(txt):
    words = [w for w in txt.split(' ') if w!='' and w not in STOPWORDS]
    return words

In [11]:
print get_keywords(txt2words2(my_document['body']))

[u'm\xe1s', u'pel\xedculas', u'chaval', u'pel\xedcula', u'personajes', u'ser', u'Harry', u'casa', u'situaci\xf3n', u'infantiles.']


## 3ª versión. Quitando mayúsculas, tildes...

In [12]:
def txt2words3(txt):
    txt = txt.lower()  # Text in lowercase
    table = dict(zip( #  Quitar tildes
        [ord(x) for x in u'áéíóúü'],
        [ord(x) for x in u'aeiouu']
    ))
    txt = txt.translate(table)    
    txt = ''.join([
        letter for letter in txt 
        if letter in set(u'abcdefghijklmnñopqrstuvwxyz0123456789 ')]
    )
    words = [w for w in txt.split(' ') if w!='' and w not in STOPWORDS]
    return words

In [13]:
print get_keywords(txt2words3(my_document['body']))

[u'pelicula', u'fan', u'casa', u'harry', u'chaval', u'luego', u'adultos', u'peliculas', u'personajes', u'potter']


## 4ª versión. Stemming

In [14]:
from nltk.stem.snowball import SpanishStemmer
stemmer = SpanishStemmer()

def txt2words4(txt):
    txt = txt.lower()  # Text in lowercase
    table = dict(zip( #  Quitar tildes
        [ord(x) for x in u'áéíóúü'],
        [ord(x) for x in u'aeiouu']
    ))
    txt = txt.translate(table)    
    txt = ''.join([
        letter for letter in txt 
        if letter in set(u'abcdefghijklmnñopqrstuvwxyz0123456789 ')]
    )
    words = [
        stemmer.stem(w)
        for w in txt.split(' ')
        if w!='' and w not in STOPWORDS
    ]
    return words

In [15]:
print get_keywords(txt2words4(my_document['body']))

[u'pelicul', u'gust', u'primer', u'personaj', u'encant', u'fan', u'magic', u'cas', u'cre', u'harry']


## Aplicándolo a todos los documentos

In [16]:
for d in documents:
    for key in ['body', 'summary']:
        d[key+'_tokens'] = txt2words4(d[key])

### Exploración

In [17]:
# Film titles
titles = sorted(list({d['title'] for d in documents}))
print '\n'.join(titles)

13 tzameti
14 Kilómetros
1408
15 días contigo
16 calles
187
2 Días en París
20.000 ańos en Sing Sing
2001, Odisea del espacio
28 semanas después
30 días de oscuridad
300
3000 millas al infierno
3:10 to Yuma
3ş colder
4 meses, 3 semanas y 2 días
53 días de invierno
8 mujeres
88 Minutos
9 songs
A golpes
A scanner darkly
A todo gas 3: Tokyo race
Abierto hasta el amanecer
Across The universe
Adaptation (El ladron de orquideas)
Adiós pequeńa, adiós
Admitido
Agitese antes de usarla
Agresión en la casa del terror
Ahora o nunca
Al final de la escapada
Al límite de la verdad
Alatriste
Alejandro Magno
Alemania ańo cero
Alfie
Algunos hombres buenos
Alien Nation
Alien Space Avenger
Alien vs Predator
Alien vs Predator 2
Alien, el octavo pasajero
Alone in the dark
Alpha Dog
Alta Tensión
Alta sociedad
Alvin y las Ardillas
Amanecer de los Muertos
American Dreamz
American Gangster
American Splendor
Amor Idiota
Amor en la tarde
Amor y otros desastres
An American Crime
Ana Karenina
Anacondas: la cacería 

In [18]:
def explore(title):
    print "Título: {}".format(title)
    for kws in [get_keywords(d['body_tokens']) for d in documents if title==d['title']]:
        print ' ->  {}'.format(', '.join(kws))

In [19]:
explore(u'Harry Potter y la piedra filosofal')

Título: Harry Potter y la piedra filosofal
 ->  pelicul, mag, demasi, primer, ser, nios, personaj, termin, represent, libr
 ->  pelicul, gust, primer, personaj, encant, fan, magic, cas, cre, harry


In [20]:
explore('High School Musical')

Título: High School Musical
 ->  pelicul, disc, nunc, gust, disney, lad, meditant, ejem, ocasion, ver
 ->  mejor, original, pelicul, school, coreografi, disney, result, juvenil, sobresalient, produccion


In [21]:
explore('Los puentes de Madison')

Título: Los puentes de Madison
 ->  amor, ser, famili, human, cas, ultim, pelicul, vid, valor, verdader


In [22]:
explore('2001, Odisea del espacio')

Título: 2001, Odisea del espacio
 ->  2001, ser, final, film, pelicul, kubrick, propi, director, llam, tan


In [23]:
explore('Alien vs Predator 2')

Título: Alien vs Predator 2
 ->  men, super, especial, rod, enfrent, falt, nuev, ser, pues, efect
 ->  predator, pelicul, original, ali, human, ver, guion, accion, cualqui, muert
 ->  ali, nunc, pelicul, demasi, predator, cutr, hech, mejor, mal, patet
 ->  predator, aliens, pelicul, ali, entreg, bastant, ident, vs, segund, requiem
 ->  pelicul, parec, mism, sal, ali, videojueg, sol, depred, aport, straus
 ->  histori, hac, version, personaj, pelicul, baj, ser, result, hech, part
 ->  pelicul, depred, ali, histori, qued, aliens, pas, medi, ahi, nav
 ->  pelicul, primer, part, dos, exact, cas, despu, aliens, depred, tod
 ->  casi, bien, carg, tont, dos, primer, ver, secuel, segund, luch
 ->  ali, pelicul, nunc, predator, pues, tierr, sal, mal, patet, human


In [24]:
explore('Apocalypto')

Título: Apocalypto
 ->  pelicul, may, nativ, gran, asi, necesari, actor, distint, principal, vez
 ->  ser, apocalypt, pelicul, gibson, may, acab, caz, persecucion, guerrer, propi
 ->  gibson, asi, histori, pelicul, mism, cuent, director, mel, jagu, cin
 ->  pelicul, histori, plan, dich, mund, gibson, aventur, estil, apocalypt, represent
 ->  violent, bien, peli, civilizacion, histor, desarroll, suficient, persecucion, rigur, ocup
 ->  son, pelicul, nuev, men, pantall, entiend, fin, histori, anterior, cont
 ->  gibson, tan, pelicul, pued, mel, jagu, salvaj, max, apocalypt, enfrent
 ->  gibson, primer, sol, parec, cin, aztec, human, pelicul, mund, realid
 ->  may, pelicul, hac, alde, pas, siend, destru, trat, grab, bien
 ->  mel, hac, indigen, consegu, interpret, cuent, sol, direccion, vist, pelicul
 ->  gust, apocalypt, cin, histori, gibson, cuent, pelicul, ultim, mejor, prim
 ->  human, cabez, tip, esper, aun, cre, vision, sub, gener, met
 ->  pelicul, cin, director, apocalypt, gibson,

## Exportar/Importar

In [25]:
import json
import zlib


# Export
with open(config.DATASET_MUCHOCINE, 'w+') as fd:
    fd.write(zlib.compress(json.dumps(documents)))
    
# Import
with open(config.DATASET_MUCHOCINE, 'r') as fd:
    docs_import =json.loads(zlib.decompress(fd.read()))